In [ ]:
from __future__ import division
import cPickle as pickle
import pandas as pd
import numpy as np
import librosa
from itertools import compress
import matplotlib.pyplot as plt
#%matplotlib inline
import thinkdsp
import thinkplot
from itertools import compress
import os
from collections import Counter


In [ ]:
def findsilence(y,sr,ind_i):
    print ("looking for silences")
    hop = int(round(sr*0.1)) #hop and width defines search window
    width = int(sr*0.1)
    n_slice = int(len(y)/hop)
    starts = np.arange(n_slice)*hop
    ends = starts+width
    if hop != width:
        cutoff = np.argmax(ends>len(y))
        starts = starts[:cutoff]
        ends = ends[:cutoff]
        n_slice = len(starts)
    
    mask = map(lambda i: np.dot(y[starts[i]:ends[i]],y[starts[i]:ends[i]])/width, range(n_slice)) > 0.2 * np.dot(y,y)/len(y)
    starts =  list(compress(starts+ind_i,mask))
    ends = list(compress(ends+ind_i,mask))
    return (starts[0],ends[len(ends)-1])

def get_sound_fragment(filename,start=0,end=None,disp=True,output=False, itr=''):
    try:
        y, sr = librosa.load(filename)
    except:
        obj = thinkdsp.read_wave(filename)
        y = obj.ys
        sr = obj.framerate
        print itr, ' : librosa.load failed for '+filename

    t = np.arange(len(y))/sr

    i = int(round(start * sr))
    if end != None:
        j = int(round(end * sr))
    else:
        j = len(y)
    startR, endR = findsilence(y[i:j],sr,i)
    start = startR/len(y)
    end = endR/len(y)
    return start, end
    
def post2df(filepathlist):
    i=0
    print (filepathlist)
    start, end = get_sound_fragment(filepathlist, disp=False, output=True, itr=i)
    return start, end

def cutFile(className, fileName, soundstartR, soundendR):
    print("reading: " + className)
    fragment = thinkdsp.read_wave(fileName)
    segment = fragment.segment(start=(soundstartR*fragment.duration), duration=(soundendR-soundstartR)*((fragment.duration)))
    segment.write(filename='Soundsimproved/cut_' + className)
    print("saved the new file")
    return

In [ ]:
oldSoundsFolderName = "./Sounds"
for className in os.listdir(oldSoundsFolderName):
    if className.endswith('.wav'):
        soundFileName = oldSoundsFolderName + "/" + className
        print("found a sound file")
        a, b = post2df(soundFileName)
        cutFile(className, soundFileName, a, b)
    else:
        print("found a non-sound file")
print ("done finding files")